In [13]:
import pandas as pd
import pickle
import re
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

EXCEL_FILE_PATH = "MI 2 Rows of Couties North of IN-OH.xlsx"
MODEL_FILE_PATH = "logreg_classifier.pickle"
VECTORIZER_FILE_PATH = "logreg_vectorizer.pickle"
FILTERED_FILE_PATH = f"{EXCEL_FILE_PATH} Classified.xlsx"
OUTPUT_FILE_PATH = f"{EXCEL_FILE_PATH} SCRUBBED.xlsx"

def load_data(file_path):
    try:
        return pd.read_excel(file_path)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return pd.DataFrame()

def clean_data(df):
    df.dropna(subset=['MAIL_ADDR', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP', 'OWNER_NAME_1'], inplace=True)
    df.drop_duplicates(subset=['MAIL_ADDR', 'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP'], inplace=True)
    df['Full Name'] = df['OWNER_NAME_1'].str.replace(',', '').str.replace('&', '')
    return df

def load_model_and_vectorizer(model_path, vectorizer_path):
    try:
        with open(model_path, "rb") as model_file:
            model = pickle.load(model_file)
        with open(vectorizer_path, "rb") as vec_file:
            vectorizer = pickle.load(vec_file)
        return model, vectorizer
    except FileNotFoundError as e:
        print(f"File not found: {e.filename}")
        return None, None

def predict_names(df, model, vectorizer):
    if 'OWNER_NAME_1' in df.columns:
        X_test = vectorizer.transform(df['OWNER_NAME_1'])
        df['Prediction'] = model.predict(X_test)
        return df
    else:
        print("The 'OWNER_NAME_1' column is missing from the Excel file.")
        return df

def save_predictions(df, file_path):
    df.to_excel(file_path, index=False)

def filter_company_names(df, human_names):
    df = df[df['OWNER_NAME_1'].isin(human_names)]
    return df

def remove_suspected_company_names(df, keywords):
    pattern = r'\b(?:' + '|'.join(map(re.escape, keywords)) + r')\b'
    df = df[~df['OWNER_NAME_1'].str.contains(pattern, case=False, na=False)]
    return df


# Main process
df = load_data(EXCEL_FILE_PATH)
df = clean_data(df)

model, vectorizer = load_model_and_vectorizer(MODEL_FILE_PATH, VECTORIZER_FILE_PATH)
if model and vectorizer:
    df = predict_names(df, model, vectorizer)
    save_predictions(df[['OWNER_NAME_1', 'Prediction']], FILTERED_FILE_PATH)
    print("Data Classified")
     
    filtered_df = load_data(FILTERED_FILE_PATH)
    human_names = filtered_df[filtered_df['Prediction'] == 'Human Name']['OWNER_NAME_1'].unique().tolist()
    df = filter_company_names(df, human_names)
    print("Data Filtered")

    # List of keywords indicating company names
    company_keywords = ['OF', 'SERVICES', 'COUNTY', 'DISTRIBUTOR', 'PRODUCTS', 'COUNTRY CLUB', 'PLLC', 'PRIVATE',
                        'COMPANY', 'INC', 'RETIREMENT', 'DEVELOPMENT', 'HOA', 'AUTHORITY', 'CONF', 'CONFERENCE',
                        'CONSTRUCTION', 'AFFORDABLE', 'HOUSING', 'MID COAST', 'ESTATE', 'REHABILITATION', 'GARDENS',
                        'WELLNESS', 'LLC', 'STATE', 'LAND', 'MEMBERSHIP', 'COOPERATIVE', 'CORP', 'CORPORATION',
                        'INDEPENDENT', 'CHURCH', 'ENTERPRISES', 'ACCOUNTING', 'INVESTMENTS', 'OWNERS', 'AIRPORT',
                        'MAINTENANCE', 'ASSOCIATION', 'REALTY', 'FOUNDATION', 'CONSULTANTS', 'ASSOCIATES',
                        'CORPORATE', 'DISTRICT', 'LTD', 'LIMITED', 'INCORPORATED', 'PROPERTIES', 'INVESTMENT',
                        'NORTHEAST', 'PLUMBING', 'HEATING', "T V A", 'New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix',
                        'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose', 'Austin', 'Jacksonville', 'Fort Worth',
                        'Columbus', 'San Francisco', 'Charlotte', 'Indianapolis', 'Seattle', 'Denver', 'Washington', 'Boston',
                        'El Paso', 'Nashville', 'Detroit', 'Oklahoma City', 'Portland', 'Las Vegas', 'Memphis', 'Louisville',
                        'Baltimore', 'Milwaukee', 'Albuquerque', 'Tucson', 'Fresno', 'Sacramento', 'Kansas City', 'Atlanta',
                        'Miami', 'Colorado Springs', 'Raleigh', 'Omaha', 'Long Beach', 'Virginia Beach', 'Oakland', 'Minneapolis',
                        'Tulsa', 'Arlington', 'Tampa', 'New Orleans', 'Wichita', 'Alabama', 'AL', 'Alaska', 'AK', 'Arizona', 'AZ',
                        'Arkansas', 'AR', 'California', 'CA', 'Colorado', 'CO', 'Connecticut', 'CT', 'Delaware', 'DE', 'Florida', 'FL',
                        'Georgia', 'GA', 'Hawaii', 'HI', 'Idaho', 'ID', 'Illinois', 'IL', 'Indiana', 'IN', 'Iowa', 'IA', 'Kansas', 'KS',
                        'Kentucky', 'KY', 'Louisiana', 'LA', 'Maine', 'ME', 'Maryland', 'MD', 'Massachusetts', 'MA', 'Michigan', 'MI',
                        'Minnesota', 'MN', 'Mississippi', 'MS', 'Missouri', 'MO', 'Montana', 'MT', 'Nebraska', 'NE', 'Nevada', 'NV',
                        'New Hampshire', 'NH', 'New Jersey', 'NJ', 'New Mexico', 'NM', 'New York', 'NY', 'North Carolina', 'NC',
                        'North Dakota', 'ND', 'Ohio', 'OH', 'Oklahoma', 'OK', 'Oregon', 'OR', 'Pennsylvania', 'PA', 'Rhode Island', 'RI',
                        'South Carolina', 'SC', 'South Dakota', 'SD', 'Tennessee', 'TN', 'Texas', 'TX', 'Utah', 'UT', 'Vermont', 'VT',
                        'Virginia', 'VA', 'Washington', 'WA', 'West Virginia', 'WV', 'Wisconsin', 'WI', 'Wyoming', 'WY',
                        'Co', 'Inc', 'LLC', 'Ltd', 'Corp', 'Pty', 'PLC', 'GmbH', 'S.A.', 'S.A.S.', 'AG', 'N.V.', 'B.V.', 'K.K.', 'S.R.L.',
                        'P.C.', 'C.A.', 'd.o.o.', 'P.L.C.', 'S.p.A.', 'A.G.', 'a.s.', 'OÜ', 'Oy', 'ApS', 's.r.o.', 'S.A.B.', 'S.L.', 'AB',
                        'CASS', 'ILE', 'BRANCH', 'ST', 'HCMA', 'ITC', 'TRANSMISSION', 'ENTERPRISES']

    df = remove_suspected_company_names(df, company_keywords)
    save_predictions(df, OUTPUT_FILE_PATH)
    print(f"Original Data Classified, Filtered, and Saved to the path {OUTPUT_FILE_PATH}")


Data Classified
Data Filtered
Original Data Classified, Filtered, and Saved to the path MI 2 Rows of Couties North of IN-OH.xlsx SCRUBBED.xlsx
